In [1]:
import pandas as pd
from pathlib import Path
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from langchain.docstore.document import Document
import pickle


ImportError: cannot import name 'FAISS' from 'langchain.vectorstores' (f:\Intelligent_Complaint_Analysis\week6\Lib\site-packages\langchain\vectorstores\__init__.py)

In [2]:
# Set up paths
VECTOR_STORE_PATH = Path('F:/Intelligent_Complaint_Analysis/data')
VECTOR_STORE_FILE = VECTOR_STORE_PATH / 'faiss_index'
EMBEDDING_MODEL_FILE = VECTOR_STORE_PATH / 'embedding_model.pkl'


In [3]:

# Load embedding model
print("Loading embedding model...")
with open(EMBEDDING_MODEL_FILE, 'rb') as f:
    embedding_model = pickle.load(f)

# Load vector store
print("Loading vector store...")
vector_store = FAISS.load_local(VECTOR_STORE_FILE, embeddings=embedding_model)

Loading embedding model...


NameError: name 'pickle' is not defined